In [1]:
import sys; sys.path.insert(0, '..')

In [2]:
from joblib import load
from scripts.get_data import da_classificare_DB
import pandas as pd
import numpy as np

In [3]:
#!/usr/bin/env python

'''
Created on Aug 1, 2016
@author: skarumbaiah
Computes Fleiss' Kappa 
Joseph L. Fleiss, Measuring Nominal Scale Agreement Among Many Raters, 1971.
'''

def fleissKappa(rate,n):
    """ 
    Computes the Kappa value
    @param rate - ratings matrix containing number of ratings for each subject per category 
    [size - N X k where N = #subjects and k = #categories]
    @param n - number of raters   
    @return fleiss' kappa
    """

    N = len(rate)
    k = len(rate[0])
    print("#raters = ", n, ", #subjects = ", N, ", #categories = ", k)
    
    #mean of the extent to which raters agree for the ith subject 
    PA = sum([(sum([i**2 for i in row])- n) / (n * (n - 1)) for row in rate])/N
    print("PA = ", PA)
    
    # mean of squares of proportion of all assignments which were to jth category
    PE = sum([j**2 for j in [sum([rows[i] for rows in rate])/(N*n) for i in range(k)]])
    print("PE =", PE)
    
    kappa = -float("inf")
    try:
        kappa = (PA - PE) / (1 - PE)
        kappa = float("{:.3f}".format(kappa))
    except ZeroDivisionError:
        print("Expected agreement = 1")

    print("Fleiss' Kappa =", kappa)
    
    return kappa

In [4]:
#! /usr/bin/env python
# -*- coding: utf-8
'''
Python implementation of Krippendorff's alpha -- inter-rater reliability
(c)2011-17 Thomas Grill (http://grrrr.org)
Python version >= 2.4 required
'''

from __future__ import print_function
try:
    import numpy as np
except ImportError:
    np = None


def nominal_metric(a, b):
    return a != b


def interval_metric(a, b):
    return (a-b)**2


def ratio_metric(a, b):
    return ((a-b)/(a+b))**2


def krippendorff_alpha(data, metric=interval_metric, force_vecmath=False, convert_items=float, missing_items=None):
    '''
    Calculate Krippendorff's alpha (inter-rater reliability):
    
    data is in the format
    [
        {unit1:value, unit2:value, ...},  # coder 1
        {unit1:value, unit3:value, ...},   # coder 2
        ...                            # more coders
    ]
    or 
    it is a sequence of (masked) sequences (list, numpy.array, numpy.ma.array, e.g.) with rows corresponding to coders and columns to items
    
    metric: function calculating the pairwise distance
    force_vecmath: force vector math for custom metrics (numpy required)
    convert_items: function for the type conversion of items (default: float)
    missing_items: indicator for missing items (default: None)
    '''
    
    # number of coders
    m = len(data)
    
    # set of constants identifying missing values
    if missing_items is None:
        maskitems = []
    else:
        maskitems = list(missing_items)
    if np is not None:
        maskitems.append(np.ma.masked_singleton)
    
    # convert input data to a dict of items
    units = {}
    for d in data:
        try:
            # try if d behaves as a dict
            diter = d.items()
        except AttributeError:
            # sequence assumed for d
            diter = enumerate(d)
            
        for it, g in diter:
            if g not in maskitems:
                try:
                    its = units[it]
                except KeyError:
                    its = []
                    units[it] = its
                its.append(convert_items(g))


    units = dict((it, d) for it, d in units.items() if len(d) > 1)  # units with pairable values
    n = sum(len(pv) for pv in units.values())  # number of pairable values
    
    if n == 0:
        raise ValueError("No items to compare.")
    
    np_metric = (np is not None) and ((metric in (interval_metric, nominal_metric, ratio_metric)) or force_vecmath)
    
    Do = 0.
    for grades in units.values():
        if np_metric:
            gr = np.asarray(grades)
            Du = sum(np.sum(metric(gr, gri)) for gri in gr)
        else:
            Du = sum(metric(gi, gj) for gi in grades for gj in grades)
        Do += Du/float(len(grades)-1)
    Do /= float(n)

    if Do == 0:
        return 1.

    De = 0.
    for g1 in units.values():
        if np_metric:
            d1 = np.asarray(g1)
            for g2 in units.values():
                De += sum(np.sum(metric(d1, gj)) for gj in g2)
        else:
            for g2 in units.values():
                De += sum(metric(gi, gj) for gi in g1 for gj in g2)
    De /= float(n*(n-1))

    return 1.-Do/De if (Do and De) else 1.


In [5]:
X = da_classificare_DB()

results = load('../scripts/supervised_dump.joblib')

classified = pd.read_excel('../dati_ceramiche_classificati.xlsx')

ModuleNotFoundError: No module named 'sklearn.neural_network.multilayer_perceptron'

In [16]:
for c in classified.iloc[:, 13:-1]:
    err = sum([x[0]-x[1] for x in zip(classified[c], classified['valutazione'])])
    print(f'{c:<18}'+':', f'{ err / len(classified) :>4.3f}')

Neural net        : 0.053
Nearest Neighbors : 0.053
SVC               : -0.015
Decision Tree     : -0.152
Random Forest     : -0.061
Naive Bayes       : -0.341
valutazione       : 0.000


In [9]:
cfs = dict()
for k, v in results.items():
    cfs[k] = max(
            zip(v['test_accuracy'], v['estimator']),
            key= lambda x: x[0]
        ) [1].best_estimator_ 

# Inter modello

In [10]:
y = []
for x in zip(*[c.predict(X) for c in cfs.values()]):
    n = sum(x)
    y.append([6-n, n])
fleissKappa(y, 6)

#raters =  6 , #subjects =  132 , #categories =  2
PA =  0.7883838383838391
PE = 0.5026814865830018
Fleiss' Kappa = 0.574


0.574

In [11]:
krippendorff_alpha([c.predict(X) for c in cfs.values()])

0.5750229203397981

In [12]:
## Calcola le medie degli stimatori, le scala secondo una parabola (y = 4x^2 -4x + 1)
# che passa per i punti: (0,1), (0.5, 0) (1, 1) e poi calcola la media di queste
#  
#
sum([4*x**2 -4*x + 1 for x in [sum(c)/6 for _, c in classified.iloc[:, -7:-1].iterrows()]])/len(classified)

NameError: name 'classified' is not defined

# Intra modello

In [20]:
split = {
    'Neural net': [],
    'Nearest Neighbors': [],
    'SVC': [],
    'Decision Tree': [],
    'Random Forest': [],
    'Naive Bayes': [],
    'valutazione': [],
}
for _, row in classified.iterrows():

    if pd.isna(row['Campione']):
        for k, v in split.items():
            v[-1].append(row[k])
    else:
        for k, v in split.items():
            v.append([row[k]])

In [21]:
# media delle deviazione standard delle classificazioni all'interno dello reperto
for k in split:
    std = np.mean([np.std(x) for x in split[k] if len(x)>1]) 
    print(f'{k:<20} -> {std:.4}')

Neural net           -> 0.07505
Nearest Neighbors    -> 0.06678
SVC                  -> 0.06366
Decision Tree        -> 0.07718
Random Forest        -> 0.08791
Naive Bayes          -> 0.06236
valutazione          -> 0.04273
